In [ ]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv(r'../input/tabular-playground-series-nov-2021/train.csv')
test = pd.read_csv(r'../input/tabular-playground-series-nov-2021/test.csv')
submission = pd.read_csv(r'../input/tabular-playground-series-nov-2021/sample_submission.csv')

In [ ]:
train.shape, test.shape, submission.shape

In [ ]:
train.head()

In [ ]:
train.drop('id',axis=1,inplace=True)
test.drop('id',axis=1,inplace=True)

### **Data Exploration:**

In [ ]:
print('train: ')
train.describe().T.style.bar(subset=['mean'], color='#606ff2')\
                            .background_gradient(subset=['std'], cmap='PuBu')\
                            .background_gradient(subset=['50%'], cmap='PuBu')

In [ ]:
print('test: ')
test.describe().T.style.bar(subset=['mean'], color='#606ff2')\
                            .background_gradient(subset=['std'], cmap='PuBu')\
                            .background_gradient(subset=['50%'], cmap='PuBu')

### **Data Preprocessing:**

In [ ]:
num_features = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 
                'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 
                'f21', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 
                'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 
                'f41', 'f42', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 
                'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 
                'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 
                'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 
                'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 
                'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99']

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
train[num_features] = ss.fit_transform(train[num_features])
test[num_features] = ss.transform(test[num_features])

### **PyCaret**

In [ ]:
! pip install pycaret

In [ ]:
from pycaret.classification import setup, compare_models, blend_models, get_config

In [ ]:
def pycaret_model(train, target,test, n_select, fold,opt):
    print('Setup Your Data....')
    setup(data=train,
          target=target,
          silent= True,use_gpu = True)
  
    print('Comparing Models....')
    best = compare_models(sort = opt,n_select=n_select, fold = fold,include = ['lightgbm','xgboost','catboost'])
    
    print('Blending Models....')
    blended = blend_models(estimator_list= best, fold=fold, optimize=opt)
    
    prep_pipe = get_config("prep_pipe")
    prep_pipe.steps.append(['final_model', blended])
    predictions = prep_pipe.predict_proba(test)[:,1]

    return predictions


In [ ]:
submission['target'] = pycaret_model(train,'target',test, 2, 5,'AUC')

In [ ]:
submission.to_csv('submission.csv',index=False)

<div class="alert alert-info">
<h4>If you like this notebook, please upvote it! 
     Thank you! :)</h4>
</div>